## Import all packages and static variable 

In [3]:
import json
import requests
from datetime import datetime, date

In [4]:
# Apps Credentials

apps_id = '5b3e1ec0'
apps_key = 'dfd9192b976d244459d36cb4a5c08039'

# requests url link format 
# {0} -> app_id, {1} -> app_key, {2} -> source, {3} -> dest, {4} -> date of departure
req_link = '''http://developer.goibibo.com/api/search/?app_id={0}&app_key={1}&format=json&source={2}&destination={3}&dateofdeparture={4}&seatingclass=E&adults=1&children=0&infants=0&counter=0'''


# 1. Extraction
## Get variable from database

In [5]:
import psycopg2
import psycopg2.extras


## First we get the config for our ETL process from db 
try:
    connect_str = "dbname='iykra_online_store' user='doadmin' port=25060 " + \
                  "host='db-postgresql-nyc1-45373-do-user-2306548-0.db.ondigitalocean.com' " + \
                  "password='bjx5qln81af941jw'"
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    # create a psycopg2 cursor that can execute queries
    cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)


# get etl configuration
cursor.execute('''SELECT id, source, dest, dod FROM public.etl_config;''')

rows = cursor.fetchall()
for row in rows:
    if row['id'] == 1:
        source = row['source']
        dest = row['dest']
        dod = row['dod']

## Extract data from flight API 
req_ready = req_link.format(apps_id, apps_key, source, dest, dod) ##beware of the paramter order 

## Hit the url to get the result
result = requests.get(req_ready)
result

'{\n    "data": {\n        "returnflights": [], \n        "onwardflights": [\n            {\n                "origin": "SIN", \n                "rating": 0, \n                "DepartureTime": "", \n                "flightcode": "263", \n                "Group": "", \n                "farebasis": "E02H01", \n                "depterminal": "4", \n                "holdflag": "", \n                "CINFO": "air-SIN-JKT-20190507--1-0-0-E-0--", \n                "deptime": "10:20", \n                "codeshare": "", \n                "ibibopartner": "airasianew", \n                "duration": "1h 55m", \n                "platingcarrier": "", \n                "qtype": "", \n                "arrterminal": "2", \n                "flightno": "263", \n                "destination": "CGK", \n                "FlHash": "oQZ-263", \n                "stops": "0", \n                "seatsavailable": "49", \n                "carrierid": "QZ", \n                "airline": "Indonesia AirAsia", \n        

In [12]:
result_rep = result.text.replace('\n','')
result_json = json.loads(result.text)

In [15]:
sum_fare = 0
total_flight = 0
for r in result_json['data']['onwardflights']:
    fare = r['fare']['totalfare']
    sum_fare += fare
    total_flight += 1

In [19]:
print(sum_fare, total_flight)
sum_fare / total_flight

4616033 146


31616.664383561645

In [4]:
## how if we want to get data for 7 days after dod in configuration
res = {}
dod_int = int(dod)
for i in range(0,8):
    dod_now = dod_int + i
    dod_str = str(dod_now)
    req_ready = req_link.format(apps_id, apps_key, source, dest, dod_str) ##beware of dod paramete 
    result = requests.get(req_ready)
    res[dod_str] = result

# 2. Transformation

## get average price for every date

In [5]:
res_json = {}
for key,r in res.items():
    result_json = json.loads(r.text) #cleans data so that can be read
    res_json[key] = result_json

## question -> what is the json structure ? 
## question -> how many records you get? 

In [6]:
fare_result = {}
for key, r_j in res_json.items():
    onward_flights = r_j['data']['onwardflights']
    dod = key
    fare_total = 0
    flight_total = 0
    for of in onward_flights:
        fare_total += int(of['fare']['totalfare'])
        flight_total += 1
    avg_fare = fare_total / max(flight_total, 1)
    fare_result[dod] = {'avg_fare': avg_fare, 'flight_total': flight_total}

# 3. Load

In [7]:
#insert data into table that has been created

query_insert = ''' insert into etl_result(source, dest, dod, avg_fare, flight_total, run_at) 
                    values ('{0}', '{1}', '{2}', {3}, {4}, '{5}') '''

tdy = datetime.strftime(datetime.now(), '%Y-%m-%d  %H:%M:%S')

for key, r_j in fare_result.items():
    query_insert_ready = query_insert.format(source, dest, key, r_j['avg_fare'], r_j['flight_total'], tdy)
    cursor.execute(query_insert_ready)
    conn.commit()



# 4. Visualize it!

In [8]:
# I'm using plotly

api_key_plotly = 'hskFfNF6BgenahVO6KFd'
username_plotly = 'sureucando'

In [10]:
import plotly 
plotly.tools.set_credentials_file(username=username_plotly, api_key=api_key_plotly)

#get data from db

query_get_data_etl = ''' select source, dest, dod, avg_fare, flight_total from public.etl_result where source = '{0}'
                        and dest = '{1}' and run_at = '{2}' '''

query_get_data_etl_ready = query_get_data_etl.format(source, dest, '2019-03-22 16:08:59')
print(query_get_data_etl_ready)

cursor.execute(query_get_data_etl_ready)

result = cursor.fetchall()

 select source, dest, dod, avg_fare, flight_total from public.etl_result where source = 'SIN'
                        and dest = 'JKT' and run_at = '2019-03-22 16:08:59' 


In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

data = {}
for row in result:
    data[row[2]] = row[3]

In [12]:

key_sorted = sorted(data)
x_data = []
y_data = []
for key in key_sorted:
    year = int(key[0:4])
    month = int(key[4:6])
    day = int(key[6:8])
    x_data.append(date(int(key[0:4]),int(key[4:6]),int(key[6:8])))
    y_data.append(data[key])

In [13]:
trace = go.Scatter(
    x = x_data,
    y = y_data
)

data = [trace]

py.iplot(data, filename='basic-line')


/usr/local/lib/python3.5/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

